In [ ]:
import pandas as pd
import numpy as np
import os
import glob

# Combine and format the transfermarkt data

Combine team files into one dataframe

In [ ]:
files = glob.glob(os.path.join('data', 'transfermarkt', 'players', '*'))
len(files)

In [ ]:
df = pd.concat([pd.read_parquet(file) for file in files])
df.reset_index(drop=True, inplace=True)
len(df)

Format jersey number

In [ ]:
df['jersey_number'] = df['jersey_number'].replace('-', None)

# You might need to edit this. As the transfermarkt value format depends on your location (mine is in euros)

In [ ]:
mask_thousand = df['market_value'].str.strip().str[-1] == 'k'
df['market_value'] = (df['market_value']
                      .replace('-', None)
                      .str.strip()
                      .str.replace('€', '', regex=False)
                      .str.replace('m', '', regex=False)
                      .str.replace('k', '', regex=False)
                     )
df['market_value'] = pd.to_numeric(df['market_value'])
df.loc[mask_thousand, 'market_value'] = df.loc[mask_thousand, 'market_value'] / 1000
df.rename({'market_value': 'market_value_euro_millions'}, axis='columns', inplace=True)

Format date/ height columns

In [ ]:
df['dob'] = pd.to_datetime(df['date_of_birth_and_age'].str.split(pat='(').str[0], errors='coerce')
df['joined'] = pd.to_datetime(df['joined'], errors='coerce')
df.drop(['date_of_birth_and_age'], axis='columns', inplace=True)
df['height'] = pd.to_numeric(df['height'].replace('-', None).str.replace('m', '').str.replace(',','.'))
df['contract'] = pd.to_datetime(df['contract'].replace('-', None))

Save the data

In [ ]:
os.makedirs(os.path.join('data', 'transfermarkt_clean'), exist_ok=True)
df.to_parquet(os.path.join('data', 'transfermarkt_clean', 'players_transfermarket.parquet'))